# This Project is for Learning Only

# Identify the Intensity by "Parselmouth" Package
In order to recognize the start and the end of each sentence we talk, we can use "paeselmouth" package to obtain intensity information through "to_inetrsity" command.  Moreover, since the defaule time step is 8ms, we can specify the time step we want in the parentheses.    

You can download the package from the link below:

https://pypi.org/project/praat-parselmouth/

To classfy start time and end time of each sentence, I design the algorithm as below:

while the "start" pointer hasn't exceeded the length of "intensity" list:

    if the voice is larger than 34dB: -> otherwise, the intersity is too low, and it may be ambient noise
        start to find the appropriate "start" point
        
        for loop -> check continuous 600 points, that is, 1ms * 600 = 0.6 sec:
                 -> the sound should last for an amount of time, otherwise, it may be immediately ambient noise
            if the start pointer exceeds the length of intensity list after adding i to it:
                then we push back the start pointer to find next appropriate start point
            if there is a point whose internsity less than 34dB:
                then we push back the start pointer to find next appropriate start point
            if i has reached 600 (the minimum time a sentence should be):
                set the current start pointer as start time
                
                while the intensity is still larger than 34dB: -> continue counting to find end point
                    if (start+i) has exceeded the length of intensity:
                        set "ending" to be "len(intensity) - 1"
                        set "start" to be "len(intersity)" to terminate the loop
                    if there is a point less than 34 dB:
                        set the very point to be "ending"
                        push back the pointer to find next appropriate start point
                    else -> the passing point is still larger than 34 dB(valid)
                        move i to next point
                calculate the start time and round it to precision 3
                calculate the ending time and round it to precision 3
                append the start time and end time to "pair" list -> in the format (start_time, end_time)
                append the duration time to "duration" list
    else:
       move i to next point

In [123]:
# Plot nice figures using Python's "standard" matplotlib library
# load the WAV file
import parselmouth
snd = parselmouth.Sound("/Users/andy/Desktop/Special_Project_second_semester/translate_file.wav")

import numpy as np
pitch = snd.to_pitch()

# find the number of the pitch
print("number of pitch = ", len(pitch.xs()))

# show the intensity of sound 
intensity = snd.to_intensity(time_step = 0.001)

# print out the length of "intensity" list -> time of amplitude
# print("length of the sampling time is:", len(intensity.xs()))

# print out the values in the "intensity" list -> amplitude in specific time point
# print("length of the amplitude is:", len(intensity.values.T))

# make sure we could continue our detection
assert len(intensity.xs()) == len(intensity.values.T)

# print(intensity.values.T)
# print(intensity.values.T(1))

# use a "empty list" to store the values in "intensity.values.T"
# timing_after = []
# voice_after = []
# timing = np.array(intensity.xs())
voice = np.array(intensity.values.T)


            
pair = []
duration = []
start = 0            
while(start<len(intensity)):
    if voice[start] >= 34:                                                  # the amplitude of voice should be larger than 40dB
        for i in range(1,601):                                              # the voice should last for a moment
                                                                            # 600 * 0.001 = 0.6 sec
            
            if start+i>= len(intensity):                                    # the pointer has exceeded the length of the intensity list
                start+=i                                                    # add i to pointer and jump out of the loop afterward
                break
            
            if voice[start+i]<34:                                           # if there is a sample less than 40dB in the 100 continuous samples
                start+=i                                                    # this segment of voice might be impulse -> discard it and find next
                break
            if i == 600:                                                    # if there are 100 continuous samples accords to this rule
                starting = start                                            # set the starting point and find ending one next
                
                while(voice[start+i]>=34):                                  # the voice should be larger than 40dB
                    
                    if start+i>= len(intensity):                            # if the pointer has reached the end of the voice
                        ending = len(intensity)-1                           # set end time as ending point
                      #  pair.append([starting,ending])
                        start = len(intensity)                              # since we have found a pair of starting time and ending one
                                                                            # we move starting pointer to the next sample and find next pair
                        break                     
                    
                    if voice[start+i+1]<34:                                 # if we find a sample less than 40dB -> this is the time voice segment ends
                        ending = start+i                                    # set the ending time to this sample
                        start += i+1                                        # do offset to start pointer
                        break
                    i+=1                                                    # If the continuous voice not ended -> check next sample
                start_time_round = round(starting*0.001, 3)
                end_time_round = round(ending*0.001, 3)
                pair.append((start_time_round, end_time_round))             # append the pair to the list
                duration.append(round((ending*0.001 - starting*0.001) ,3))  # append the duration pair to the duration list
    
    else:
        start+=1

number of pitch =  15417


In [124]:
# show the content of pair list
pair

[(0.016, 8.054),
 (9.15, 10.754),
 (11.102, 14.011),
 (14.49, 18.787),
 (19.332, 22.501),
 (23.234, 26.547),
 (26.844, 28.152),
 (28.271, 29.618),
 (29.889, 33.666),
 (33.685, 36.338),
 (36.89, 38.584),
 (38.6, 41.094),
 (41.855, 47.184),
 (47.776, 50.046),
 (50.93, 53.569),
 (53.614, 54.416),
 (54.957, 59.028),
 (59.786, 60.745),
 (61.083, 63.273),
 (63.33, 65.492),
 (66.055, 66.783),
 (66.999, 69.893),
 (71.629, 72.951),
 (73.853, 75.797),
 (77.349, 79.251),
 (80.289, 81.084),
 (81.104, 83.202),
 (83.281, 89.229),
 (89.273, 90.152),
 (90.551, 93.533),
 (94.659, 96.831),
 (97.069, 100.827),
 (101.035, 101.665),
 (101.79, 102.753),
 (102.787, 103.523),
 (104.203, 108.645),
 (109.59, 111.1),
 (111.649, 113.237),
 (113.497, 116.37),
 (116.404, 117.838),
 (118.768, 124.611),
 (125.526, 126.782),
 (127.067, 128.765),
 (128.789, 130.918),
 (130.976, 131.929),
 (132.283, 133.973),
 (134.392, 136.571),
 (137.205, 137.831),
 (137.898, 142.844),
 (142.952, 143.862),
 (144.434, 153.972)]

In [125]:
# show the length of pair list
len(pair)

51

In [126]:
# show the content of duration list
duration

[8.038,
 1.604,
 2.909,
 4.297,
 3.169,
 3.313,
 1.308,
 1.347,
 3.777,
 2.653,
 1.694,
 2.494,
 5.329,
 2.27,
 2.639,
 0.802,
 4.071,
 0.959,
 2.19,
 2.162,
 0.728,
 2.894,
 1.322,
 1.944,
 1.902,
 0.795,
 2.098,
 5.948,
 0.879,
 2.982,
 2.172,
 3.758,
 0.63,
 0.963,
 0.736,
 4.442,
 1.51,
 1.588,
 2.873,
 1.434,
 5.843,
 1.256,
 1.698,
 2.129,
 0.953,
 1.69,
 2.179,
 0.626,
 4.946,
 0.91,
 9.538]

In [127]:
# show the length of duration list
len(duration)

51

# SRT File Generating
The SRT file should include three kind of information:
1. index of sentence
2. (start_time) --> (end_time)
3. content of sentence

In [128]:
# generate index list
# the index is appended in "string" type
number = []
for i in range(len(duration)):
    number.append(str(i+1))

In [129]:
# show the index list
number

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51']

In [130]:
# generate the timing pair
hour_pair = []
minute_pair = []
second_pair = []
for i in range(len(pair)):
    if (pair[i][0] >= 3600) and (pair[i][1] >= 3600):                           # case1: both start and end exceed 60 minutes
        # store the original data first
        temp1 = pair[i][0]
        temp2 = pair[i][1]
        
        # process the hour part
        temp_hour_start = temp1 / 3600
        temp_hour_end = temp2 / 3600
        temp1 = temp1 - temp_hour_start*3600                                    # minus the hour part from original data
        temp2 = temp2 - temp_hour_end*3600                                      # minus the hour part from original data
        if(temp_hour_start < 10) and (temp_hour_end < 10):
            hour_pair.append(('0' + str(temp_hour_start), '0' + str(temp_hour_end)))
        elif(temp_hour_start < 10) and (temp_hour_end > 10):
            hour_pair.append(('0' + str(temp_hour_start), str(temp_hour_end)))
        else:
            hour_pair.append((str(temp_hour_start), str(temp_hour_end)))
            
        
        # process the minute part
        temp_minute_start = temp1 / 60
        temp_minute_end = temp2 / 60
        temp1 = temp1 - temp_minute_start*60                                    # minus the minute part from original data
        temp2 = temp2 - temp_minute_end*60                                      # minus the minute part from original data
        if(temp_minute_start < 10) and (temp_minute_end < 10):
            minute_pair.append(('0' + str(temp_minute_start), '0' + str(temp_minute_end)))
        elif(temp_minute_start < 10) and (temp_minute_end > 10):
            minute_pair.append(('0' + str(temp_minute_start), str(temp_minute_end)))
        else:
            minute_pair.append((str(temp_minute_start), str(temp_minute_end)))

        
        # process the second part
        temp1_int = int(temp1)                                                  # take the second part from original data
        temp2_int = int(temp2)                                                  # take the second part from original data
        temp1 = str(int((temp1 - int(temp1))*1000))                             # only the time less than a second left
        temp2 = str(int((temp2 - int(temp2))*1000))                             # only the time less than a second left
        
        # adjust the representation method of each digit
        # otherwise, it cannot fit the format of SRT file
        # for example, "1:23:12,800" is invalid, it should be revised as "01:23:12,800"
        #              "01:22:11.900" is invalid, it should be revised as "01:22:11,900"
        if(temp1_int < 10) and (temp2_int < 10): 
            second_pair.append(('0' + str(temp1_int) + "," + temp1, '0' + str(temp2_int) + "," + temp2))
        elif(temp_minute_start < 10) and (temp_minute_end > 10):
            second_pair.append(('0' + str(temp1_int) + "," + temp1, str(temp2_int) + "," + temp2))
        else:
            second_pair.append((str(temp1_int) + "," + temp1, str(temp2_int) + "," + temp2))
        
    elif (pair[i][0] < 3600) and (pair[i][1] >= 3600):                          # case2: only ending point exceed 60 minutes
        # store the original data first
        temp1 = pair[i][0]
        temp2 = pair[i][1]
        
        # process the hour part
        #temp_hour_start = temp1 / 3600
        temp_hour_end = temp2 / 3600
        #temp1 = temp1 - temp_hour_start*3600                                   # minus the hour part from original data
        temp2 = temp2 - temp_hour_end*3600                                      # minus the hour part from original data
        if(temp_hour_start < 10) and (temp_hour_end < 10):
            hour_pair.append(('0' + str(temp_hour_start), '0' + str(temp_hour_end)))
        elif(temp_hour_start < 10) and (temp_hour_end > 10):
            hour_pair.append(('0' + str(temp_hour_start), str(temp_hour_end)))
        else:
            hour_pair.append((str(temp_hour_start), str(temp_hour_end)))
        
        # process the minute part
        temp_minute_start = temp1 / 60
        temp_minute_end = temp2 / 60
        temp1 = temp1 - temp_minute_start*60                                    # minus the minute part from original data
        temp2 = temp2 - temp_minute_end*60                                      # minus the minute part from original data
        if(temp_minute_start < 10) and (temp_minute_end < 10):
            minute_pair.append(('0' + str(temp_minute_start), '0' + str(temp_minute_end)))
        elif(temp_minute_start < 10) and (temp_minute_end > 10):
            minute_pair.append(('0' + str(temp_minute_start), str(temp_minute_end)))
        else:
            minute_pair.append((str(temp_minute_start), str(temp_minute_end)))
        
        # process the second part
        temp1_int = int(temp1)                                                  # take the second part from original data
        temp2_int = int(temp2)                                                  # take the second part from original data
        temp1 = str(int((temp1 - int(temp1))*1000))                                        # only the time less than a second left
        temp2 = str(int((temp2 - int(temp2))*1000))                                     # only the time less than a second left
        if(temp1_int < 10) and (temp2_int < 10):
            second_pair.append(('0' + str(temp1_int) + "," + temp1, '0' + str(temp2_int) + "," + temp2))
        elif(temp_minute_start < 10) and (temp_minute_end > 10):
            second_pair.append(('0' + str(temp1_int) + "," + temp1, str(temp2_int) + "," + temp2))
        else:
            second_pair.append((str(temp1_int) + "," + temp1, str(temp2_int) + "," + temp2))
            
    else:                                                                       # case3: both less than 60 minutes 
        # store the original data first
        temp1 = pair[i][0]
        temp2 = pair[i][1]
        
        # process the hour part
        hour_pair.append(('00', '00'))
        
        # process the minute part
        temp_minute_start = int(temp1/60)
        temp_minute_end = int(temp2/60)
        temp1 = temp1 - temp_minute_start*60                                    # minus the minute part from original data
        temp2 = temp2 - temp_minute_end*60                                      # minus the minute part from original data
        if(temp_minute_start < 10) and (temp_minute_end < 10):
            minute_pair.append(('0' + str(temp_minute_start), '0' + str(temp_minute_end)))
        elif(temp_minute_start < 10) and (temp_minute_end > 10):
            minute_pair.append(('0' + str(temp_minute_start), str(temp_minute_end)))
        else:
            minute_pair.append((str(temp_minute_start), str(temp_minute_end)))
        
        # process the second part
        temp1_int = int(temp1)                                                  # take the second part from original data
        temp2_int = int(temp2)                                                  # take the second part from original data
        temp1 = str(int((temp1 - int(temp1))*1000))                                        # only the time less than a second left
        temp2 = str(int((temp2 - int(temp2))*1000))                                     # only the time less than a second left
        if(temp1_int < 10) and (temp2_int < 10):
            second_pair.append(('0' + str(temp1_int) + "," + temp1, '0' + str(temp2_int) + "," + temp2))
        elif(temp1_int < 10) and (temp2_int >= 10):
            second_pair.append(('0' + str(temp1_int) + "," + temp1, str(temp2_int) + "," + temp2))
        else:
            second_pair.append((str(temp1_int) + "," + temp1, str(temp2_int) + "," + temp2))

In [131]:
# show the hour list
hour_pair

[('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00')]

In [132]:
# show the minute list
minute_pair

[('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '00'),
 ('00', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '01'),
 ('01', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02'),
 ('02', '02')]

In [133]:
# show the second list
second_pair

[('00,16', '08,54'),
 ('09,150', '10,753'),
 ('11,102', '14,10'),
 ('14,490', '18,786'),
 ('19,332', '22,501'),
 ('23,234', '26,547'),
 ('26,844', '28,152'),
 ('28,271', '29,617'),
 ('29,888', '33,665'),
 ('33,685', '36,338'),
 ('36,890', '38,584'),
 ('38,600', '41,94'),
 ('41,854', '47,183'),
 ('47,776', '50,45'),
 ('50,929', '53,569'),
 ('53,613', '54,415'),
 ('54,957', '59,27'),
 ('59,786', '0,744'),
 ('01,82', '03,273'),
 ('03,329', '05,492'),
 ('06,55', '06,783'),
 ('06,998', '09,893'),
 ('11,629', '12,950'),
 ('13,852', '15,796'),
 ('17,349', '19,251'),
 ('20,289', '21,84'),
 ('21,103', '23,201'),
 ('23,281', '29,228'),
 ('29,272', '30,152'),
 ('30,551', '33,533'),
 ('34,659', '36,831'),
 ('37,69', '40,826'),
 ('41,34', '41,665'),
 ('41,790', '42,753'),
 ('42,787', '43,522'),
 ('44,203', '48,644'),
 ('49,590', '51,99'),
 ('51,649', '53,236'),
 ('53,496', '56,370'),
 ('56,403', '57,837'),
 ('58,768', '4,611'),
 ('05,525', '06,781'),
 ('07,66', '08,764'),
 ('08,788', '10,918'),
 ('

In [134]:
# declare a list to record start_time only 
offset_start_time = []
offset = pair

In [135]:
# since the beginning and endding segment of video may be "music only"
# remove the first term and the last term
offset = offset[1:len(offset)-1]
duration = duration[1:len(duration)-1]

In [136]:
# append the start time of each pair to offset_start_time list
for i in range(len(offset)):
    offset_start_time.append(offset[i][0])

Next, we use "Google Recognizer" to translate the content of video

In [137]:
# start to do the speech recognition
# we use "speech_recognition" and "google recognizer" to do the translate
import speech_recognition as sr


recognizor = sr.Recognizer()
intro = sr.AudioFile('translate_file.wav')
audio = []
google_recognizor = []

In [138]:
# construct the audio and google_recognizer lists by appending sr.Recognizer() related information
for i in range(len(offset_start_time)):
    with intro as source:
        audio_element = recognizor.record(source, offset = offset_start_time[i], duration = duration[i]) 
        audio.append(audio_element)
for i in range(len(offset_start_time)):     
    google_recognizor_element = recognizor.recognize_google(audio[i], language = "zh-tw", show_all = True)
    google_recognizor.append(google_recognizor_element)

In [139]:
# show the translated content stored in "google_recognizer" list
google_recognizor

[{'alternative': [{'transcript': '各位同學大家好', 'confidence': 0.91268235},
   {'transcript': '各位同學大家'}],
  'final': True},
 {'alternative': [{'transcript': '我是清華大學獸醫系的顏東勇教授', 'confidence': 0.89615387},
   {'transcript': '我是清華大學數學系的顏東勇教授'},
   {'transcript': '我是清華大學數位系的顏東勇教授'},
   {'transcript': '我是清華大學數理系的顏東勇教授'},
   {'transcript': '我是清華大學數位系地顏東勇教授'}],
  'final': True},
 {'alternative': [{'transcript': '非常高興可以跟大家一同在線上學習維基',
    'confidence': 0.90104336},
   {'transcript': '非常高興可以跟大家一同在線上學習危機'},
   {'transcript': '非常高興可以跟大家一同載線上學習維基'},
   {'transcript': '非常高興可以跟大家一同在旋上學習為機'},
   {'transcript': '非常高興可以跟大家一同在線上學習為機'}],
  'final': True},
 {'alternative': [{'transcript': '一下我們來談談這門課的學習目標', 'confidence': 0.92482752},
   {'transcript': '以下我們來談談這門課的學習目標'},
   {'transcript': '1下我們來談談這門課的學習目標'}],
  'final': True},
 {'alternative': [{'transcript': '正文課將會學習微積分最基本的', 'confidence': 0.90594834},
   {'transcript': '鄭文課將會學習微積分最基本的'},
   {'transcript': '正文課江蕙學習微積分最基本的'},
   {'transcript': '鄭文克將會學習微積分最基本的'},

In [121]:
# declare a list to store the final content
content = []

In [122]:
# start to choose the content we want and append it to "content" list
for i in range(len(google_recognizor)):
    subtitle = google_recognizor[i].get("alternative")[0].get("transcript")
    content.append(subtitle)

AttributeError: 'list' object has no attribute 'get'

In [109]:
# show the final content
content

['各位同學大家好',
 '我是清華大學獸醫系的顏東勇教授',
 '非常高興可以跟大家一同在線上學習維基',
 '一下我們來談談這門課的學習目標',
 '正文課將會學習微積分最基本的',
 '總共分四章',
 '第一章集',
 '這一章將是所有初學微積分地圖',
 '所需面對最難學習的',
 '在學習極限的時候',
 '我們將從最基本的定義',
 '在學習的過程當中我們將與高中所學的學過的極限',
 '做一次性的講解一筆',
 '然後推倒一些極限的基本',
 '杏子',
 '在這一張最後我們將談到所謂的連續函',
 '第二章',
 '我要談的是為1分',
 '首先介紹微分的定義',
 '然後',
 '推倒一些微分的基本公式與行',
 '譬如項圈入',
 '三角函數的微分工',
 '因數的微分等等',
 '第三章',
 '首先學習君子的',
 '然後利用均值定理學習如何應用導函數去找一個函數的極大值',
 '取消',
 '並且找一個函數的遞增區間',
 '當我們有了這些資訊以後',
 '我們就可以很容易的畫出一個函數的圖形圖']

In [110]:
len(content)

31

In [111]:
# since the beginning and endding segments are music only
# append "music" to original list
new_content = []
new_content.append('music')
for i in range(len(content)):
    new_content.append(content[i])
new_content.append('music')
content = new_content

In [112]:
# show the new content list
content

['music',
 '各位同學大家好',
 '我是清華大學獸醫系的顏東勇教授',
 '非常高興可以跟大家一同在線上學習維基',
 '一下我們來談談這門課的學習目標',
 '正文課將會學習微積分最基本的',
 '總共分四章',
 '第一章集',
 '這一章將是所有初學微積分地圖',
 '所需面對最難學習的',
 '在學習極限的時候',
 '我們將從最基本的定義',
 '在學習的過程當中我們將與高中所學的學過的極限',
 '做一次性的講解一筆',
 '然後推倒一些極限的基本',
 '杏子',
 '在這一張最後我們將談到所謂的連續函',
 '第二章',
 '我要談的是為1分',
 '首先介紹微分的定義',
 '然後',
 '推倒一些微分的基本公式與行',
 '譬如項圈入',
 '三角函數的微分工',
 '因數的微分等等',
 '第三章',
 '首先學習君子的',
 '然後利用均值定理學習如何應用導函數去找一個函數的極大值',
 '取消',
 '並且找一個函數的遞增區間',
 '當我們有了這些資訊以後',
 '我們就可以很容易的畫出一個函數的圖形圖',
 'music']

In the end, after processing the data as above, we can restore the result by file in out

In [ ]:
# save the content to a srt file
f = open("translation.srt", "w+")
for i in range(len(content)):
    f.write(number[i])
    f.write("\n")
    f.write(hour_pair[i][0] + ':' + minute_pair[i][0] + ':' + second_pair[i][0] + " --> " + \
           hour_pair[i][1] + ':' + minute_pair[i][1] + ':' + second_pair[i][1])
    f.write("\n")  
    f.write(content[i])
    f.write("\n\n")
f.close()

In [ ]:
# we can also print the content we write in the SRT file
for i in range(len(content)):
    print(number[i])
    print(hour_pair[i][0] + ':' + minute_pair[i][0] + ':' + second_pair[i][0] + " --> " + \
           hour_pair[i][1] + ':' + minute_pair[i][1] + ':' + second_pair[i][1])
    print(content[i])  